In [ ]:
import torch
import torch.nn as nn

In [16]:
# lstm & lstmp 源码
# 常量：batchsize, time, inputsizt, hiddensize
bs, T, i_size, h_size = 2, 3, 4, 5
# proj_size = 
input = torch.randn(bs, T, i_size)  # 正态分布 输入序列
c0 = torch.randn(bs, h_size)        # 只考虑一层，初始值，不参与训练
h0 = torch.randn(bs, h_size)        # 初始值

# 调用官方API
lstm_layer = nn.LSTM(i_size, h_size, batch_first=True)
output, (h_final, c_final) = lstm_layer(input, (h0.unsqueeze(0), c0.unsqueeze(0)))       # 输入里的元组维度是三维
print(output)
# print(output)

# for k, v in lstm_layer.named_parameters():
#     print(k, v.shape)
# 打印后可以看到有4个参数，是拼接的
'''
weight_ih_l0 torch.Size([20, 4])    对应各个w_i拼接 与input相乘 h_size*4, i_size
weight_hh_l0 torch.Size([20, 5])    对应各个w_h拼接 与上一时刻隐藏状态相乘 h_size*4, h_size
bias_ih_l0 torch.Size([20])         对应b_i
bias_hh_l0 torch.Size([20])         对应b_h
'''





tensor([[[ 0.1146, -0.2298, -0.0572,  0.0379, -0.1007],
         [ 0.0352, -0.2922,  0.0129,  0.0849, -0.0091],
         [ 0.0983, -0.2999,  0.0324,  0.1157, -0.1778]],

        [[-0.2604,  0.1363, -0.0916, -0.0806, -0.2363],
         [-0.0607, -0.0717, -0.0270, -0.0547, -0.0796],
         [-0.1115,  0.0628, -0.0180,  0.0091,  0.0401]]],
       grad_fn=<TransposeBackward0>)


'\nweight_ih_l0 torch.Size([20, 4])    对应各个w_i拼接 与input相乘 h_size*4, i_size\nweight_hh_l0 torch.Size([20, 5])    对应各个w_h拼接 与上一时刻隐藏状态相乘 h_size*4, h_size\nbias_ih_l0 torch.Size([20])         对应b_i\nbias_hh_l0 torch.Size([20])         对应b_h\n'

![image-20220903161843731](123/image-20220903161843731.png)

In [14]:
# 手撸版
def lstm_forward(input, initial_state, w_ih, w_hh, b_ih, b_hh):
    h0, c0 = initial_state  # 初始状态
    bs, T, i_size = input.shape
    h_szie = w_ih.shape[0] // 4
    
    prev_h = h0
    prev_c = c0
    # 扩维
    batch_w_ih = w_ih.unsqueeze(0).tile(bs, 1, 1)  # [bs, 4*h_size, i_size]
    batch_w_hh = w_hh.unsqueeze(0).tile(bs, 1, 1)  # [bs, 4*h_size, h_size]

    output_size = h_size
    output = torch.zeros(bs, T, output_size)    # 输出矩阵

    for t in range(T):
        x = input[:, t, :] # 当前时刻输入向量 [bs, i_size] 最后要扩一维
        w_times_x = torch.bmm(batch_w_ih, x.unsqueeze(-1))   # [bs, 4*h_size, 1]
        w_times_x = w_times_x.squeeze(-1)  # [bs, 4*h_size]

        w_times_h_prev = torch.bmm(batch_w_hh, prev_h.unsqueeze(-1))
        w_times_h_prev = w_times_h_prev.squeeze(-1)
        
        # 分别计算输入门i、遗忘门f、cell门g、输出门o
        i_t = torch.sigmoid(w_times_x[:, :h_size] + w_times_h_prev[:, :h_size] + b_ih[:h_size] + b_hh[:h_size])
        f_t = torch.sigmoid(w_times_x[:, h_size:2*h_size] + w_times_h_prev[:, h_size:2*h_size] +\
             b_ih[h_size:2*h_size] + b_hh[h_size:2*h_size])
        g_t = torch.tanh(w_times_x[:, 2*h_size:3*h_size] + w_times_h_prev[:, 2*h_size:3*h_size] +\
             b_ih[2*h_size:3*h_size] + b_hh[2*h_size:3*h_size])
        o_t = torch.sigmoid(w_times_x[:, 3*h_size:4*h_size] + w_times_h_prev[:, 3*h_size:4*h_size] +\
             b_ih[3*h_size:4*h_size] + b_hh[3*h_size:4*h_size])

        # 更新
        prev_c = f_t * prev_c + i_t * g_t
        prev_h = o_t * torch.tanh(prev_c)

        output[:, t, :] = prev_h

    return output, (prev_h,prev_c)




In [18]:
# 对比验证
print(output)
# 将参数直接放入
output1, (h_final1, c_final1) = lstm_forward(input, (h0, c0), lstm_layer.weight_ih_l0, lstm_layer.weight_hh_l0, lstm_layer.bias_ih_l0, lstm_layer.bias_hh_l0)
print(output1)
# 相同！

tensor([[[ 0.1146, -0.2298, -0.0572,  0.0379, -0.1007],
         [ 0.0352, -0.2922,  0.0129,  0.0849, -0.0091],
         [ 0.0983, -0.2999,  0.0324,  0.1157, -0.1778]],

        [[-0.2604,  0.1363, -0.0916, -0.0806, -0.2363],
         [-0.0607, -0.0717, -0.0270, -0.0547, -0.0796],
         [-0.1115,  0.0628, -0.0180,  0.0091,  0.0401]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.1146, -0.2298, -0.0572,  0.0379, -0.1007],
         [ 0.0352, -0.2922,  0.0129,  0.0849, -0.0091],
         [ 0.0983, -0.2999,  0.0324,  0.1157, -0.1778]],

        [[-0.2604,  0.1363, -0.0916, -0.0806, -0.2363],
         [-0.0607, -0.0717, -0.0270, -0.0547, -0.0796],
         [-0.1115,  0.0628, -0.0180,  0.0091,  0.0401]]], grad_fn=<CopySlices>)


In [2]:
import torch
import torch.nn as nn

class Net (nn.Module):
    def __init__(self):             # 初始化
        super().__init__()
    
    def forward(self, input):       # 只需要正向传播，反向传播不用自己写
        output = input + 1
        return output               # 就是整个网络的返回值

net = Net()
x = torch.tensor(1.0)
y = net(x)
print(y)

tensor(2.)
